# Convolutional Autoencoder

In [1]:
from theano import tensor as T
import theano.tensor.nnet as nnet
import theano
import numpy as np

floatX = theano.config.floatX

def conv2d(X, kernal_shape):
    assert len(kernal_shape) == 4, "We need a 4D tensor"
    K = theano.shared(np.random.randn(*kernal_shape).astype(floatX))
    return nnet.conv2d(X, K), K

def deconv2d(X, kernal_shape):
    assert len(kernal_shape) == 4, "We need a 4D tensor"
    K = theano.shared(np.random.randn(*kernal_shape).astype(floatX))
    dummy_image = T.zeros_like(X)
    C = nnet.conv2d(dummy_image, K)
    return T.grad(T.sum(C), wrt=dummy_image, known_grads={C: X}), K

x = np.random.randn(1, 1, 3, 3)
X = T.dtensor4()

In [2]:
Y, K = conv2d(X, (1, 1, 2, 2))
Y.eval({X: x})

array([[[[-1.82317023,  0.6997941 ],
         [-1.79145671,  2.88682959]]]])

In [3]:
Y, K = deconv2d(X, (1, 1, 2, 2))
Y.eval({X: x})

array([[[[-1.30273398,  3.61756764,  1.22292444,  0.44044936],
         [-0.7041309 , -2.41066614, -2.79238586,  1.13051184],
         [-0.30588658,  3.90410377, -4.10935702,  1.02847354],
         [-0.4573692 ,  0.32098561,  1.14192934, -0.59506591]]]])

In [9]:
X = T.dtensor4()
Y, K1 = conv2d(X, (1, 1, 2, 2))
Y, K2 = conv2d(Y, (1, 1, 3, 3))
Y, K3 = deconv2d(Y, (1, 1, 3, 3))
Y, K4 = deconv2d(Y, (1, 1, 2, 2))
kernals = [K1, K2, K3, K4]

In [10]:
step_size = 0.01
meanSquaredError = T.mean(T.sqr(Y - X))
updates = [(k, k - T.grad(meanSquaredError, wrt=k) * step_size) for k in kernals]

In [11]:
train_model = theano.function(inputs=[X], outputs=[meanSquaredError], updates=updates)
evaluate_model = theano.function(inputs=[X], outputs=[Y])

In [15]:
size = 5
ident_matrix = np.eye(size).reshape(1, 1, size, size)
for i in range(100000):
    train_model(ident_matrix)
    
y = evaluate_model(ident_matrix)

In [16]:
np.round(y, 1)

array([[[[[ 0.8,  0. , -0. , -0. , -0.3],
          [ 0. ,  1. , -0. ,  0. , -0. ],
          [-0. , -0. ,  1.2, -0. , -0. ],
          [-0. ,  0. , -0. ,  1. ,  0. ],
          [-0.3, -0. , -0. ,  0. ,  0.8]]]]])

In [17]:
np.round(y).astype('int')

array([[[[[1, 0, 0, 0, 0],
          [0, 1, 0, 0, 0],
          [0, 0, 1, 0, 0],
          [0, 0, 0, 1, 0],
          [0, 0, 0, 0, 1]]]]])